<!-- TABS -->
# Answer question with LLM

In [ ]:
# <tab: No-context>

llm.predict_one(query)

In [ ]:
# <tab: Prompt>
from pinnacledb import objectmodel
from pinnacledb.components.graph import Graph, input_node

@objectmodel
def build_prompt(query):
    return f"Translate the sentence into German: {query}"

in_ = input_node('query')
prompt = build_prompt(query=in_)
answer = llm(X=prompt)
prompt_llm = answer.to_graph("prompt_llm")
prompt_llm.predict_one(query)[0]

In [ ]:
# <tab: Context>
from pinnacledb import objectmodel
from pinnacledb.components.graph import Graph, input_node

prompt_template = (
    "Use the following context snippets, these snippets are not ordered!, Answer the question based on this context.\n"
    "{context}\n\n"
    "Here's the question: {query}"
)


@objectmodel
def build_prompt(query, docs):
    chunks = [doc["text"] for doc in docs]
    context = "\n\n".join(chunks)
    prompt = prompt_template.format(context=context, query=query)
    return prompt
    

in_ = input_node('query')
vector_search_results = vector_search_model(query=in_)
prompt = build_prompt(query=in_, docs=vector_search_results)
answer = llm(X=prompt)
context_llm = answer.to_graph("context_llm")
context_llm.predict_one(query)